## Prerequisite

- Install *MATLAB engine API*
- Update *VideoUtils* under *${Human3.6M code}/external_utils/VideoUtils_v1_2/*
- Change all MATLAB classes to inherit *handle* class

In [1]:
from functools import lru_cache
import imageio
import matlab.engine
import numpy as np
from os import path, walk
from os.path import abspath, curdir, exists, basename
import cv2
import copy
import math
from tqdm import tqdm as tqdm

In [2]:
class H36M(object):
    '''Parser for Human3.6M.
    
    With MATLAB engine, this class runs Human3.6M MATLAB scripts and gets the data.
    
    Attrib:
        root: The path to Human3.6M MATLAB scripts directory.
        script_paths: The sub-directory paths excluding git directory.
        additional_script_paths: The paths to the customized MATLAB scripts.
        
        _core: The MATLAB engine object.
    '''
    root = abspath(path.join('D:', 'data', 'Human3.6M', 'Release-v1.1'))
    script_paths = [subdir for subdir, _, _ in walk(root) if '.git' not in subdir]
    additional_script_paths = [
    ]
    subjects = [1, 5, 6, 7, 8, 9, 11]
    
    def __init__(self):
        '''Initialize the MATLAB engine and preload common instances.
        '''
        
        # Initialize the MATLAB engine object.
        self._core = matlab.engine.start_matlab()
        for script_path in H36M.script_paths + H36M.additional_script_paths:
            self._core.addpath(script_path)

        # Preload H36MDataBase and necessary metadata of annotations.
        self._core.workspace['DB'] = self._core.H36MDataBase.instance()
        self._core.workspace['feature_RGB'] = self._core.H36MRGBVideoFeature()
        self._core.workspace['feature_BB'] = self._core.H36MMyBBMask()
        self._core.workspace['feature_BG'] = self._core.H36MMyBGMask()
        self._core.workspace['features'] = [
            # self._core.H36MPose3DPositionsFeature(),
            # self._core.H36MPose2DPositionsFeature(), # Use Camera.project()
            self._core.H36MPose3DPositionsFeature('Monocular', True), # Use TransformJointsPosition()
        ]
    
    @property
    def workspace(self):
        return self._core.workspace
    
    def __call__(self, inst, exe_only=False):
        if exe_only:
            return self._core.eval(inst, nargout=0)
        return self._core.eval(inst)
    
    def RGB_image(self, subject, action, sub_action, camera, frame):
        '''Get a single RGB image.
        
        Params:
            subject: The subject number in 1, 5, 6, 7, 8, 9 and 11.
            action: The action number in the range between 1 and 16.
            sub_action: The sub-action number 1 or 2.
            camera: The camera number in the range between 1 and 4.
            frame: The frame number.
        
        Return:
            RGB image as numpy array in the height-width-channel shape.
        '''
        if not self.valid_sequence(subject, action, sub_action, camera):
            raise IndexError()
        
        max_frame = self.max_frame(subject, action, sub_action)
        if not (1 <= frame <= max_frame):
            raise IndexError()
        
        sequence = self._sequence(subject, action, sub_action, camera)
        RGB = self._RGB_handle(subject, action, sub_action, camera)
        image = self._core.getFrame(RGB, self._core.double(frame))
        image = np.reshape(np.asarray(image._data, dtype=np.float), newshape=(image._size[2], image._size[1], image._size[0])).transpose(2, 1, 0)

        return image
    
    def camera_intrinsics(self, subject, action, sub_action, camera, raw_data=False):
        
        if not self.valid_sequence(subject, action, sub_action, camera):
            raise IndexError()
            
        seq = self._sequence(subject, action, sub_action, camera)
        self.workspace['tmp'] = self._core.getCamera(seq)

        f, c, k, p = [self._core.eval('tmp.' + prop)[0] for prop in ['f', 'c', 'k', 'p']]

        self._core.eval('clear tmp;', nargout=0)
        
        if raw_data:
            return f, c, k, p

        distort = np.asarray([k[0], k[1], p[0], p[1], k[2]])
        cam_mat = np.mat('%f 0 %f; 0 %f %f; 0 0 1' % (f[0], c[0], f[1], c[1]))
        
        return distort, cam_mat
    
    def undistorted_RGB_image(self, subject, action, sub_action, camera, frame):
        
        if not self.valid_sequence(subject, action, sub_action, camera):
            raise IndexError()
            
        distort, cam_mat = self.camera_intrinsics(subject, action, sub_action, camera)
        image = self.RGB_image(subject, action, sub_action, camera, frame)
        
        h,  w = image.shape[:2]
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(cam_mat,distort,(w,h),1,(w,h))

        # undistort
        dst = cv2.undistort(image, cam_mat, distort, None, newcameramtx)

        # crop the image
        x,y,w,h = roi
        dst = dst[y:y+h, x:x+w]
        
        return dst
    
    def BB_mask(self, subject, action, sub_action, camera, frame):
        '''Get a single bounding-box mask.
        
        Params:
            subject: The subject number in 1, 5, 6, 7, 8, 9 and 11.
            action: The action number in the range between 1 and 16.
            sub_action: The sub-action number 1 or 2.
            camera: The camera number in the range between 1 and 4.
            frame: The frame number.
        
        Return:
            The bounding-box mask as numpy array in the height-width shape.
        '''
        if not self.valid_sequence(subject, action, sub_action, camera):
            raise IndexError()
        
        max_frame = self.max_frame(subject, action, sub_action)
        if not (1 <= frame <= max_frame):
            raise IndexError()
        
        sequence = self._sequence(subject, action, sub_action, camera)
        BB = self._BB_handle(subject, action, sub_action, camera)
        mask = self._core.getFrame(BB, self._core.double(frame))
        mask = np.reshape(np.asarray(mask._data, dtype=np.float), newshape=(mask._size[1], mask._size[0])).transpose(1, 0)

        return mask
    
    def BG_mask(self, subject, action, sub_action, camera, frame):
        '''Get a single background mask.
        
        Params:
            subject: The subject number in 1, 5, 6, 7, 8, 9 and 11.
            action: The action number in the range between 1 and 16.
            sub_action: The sub-action number 1 or 2.
            camera: The camera number in the range between 1 and 4.
            frame: The frame number.
        
        Return:
            The background mask as numpy array in the height-width shape.
        '''
        if not self.valid_sequence(subject, action, sub_action, camera):
            raise IndexError()
        
        max_frame = self.max_frame(subject, action, sub_action)
        if not (1 <= frame <= max_frame):
            raise IndexError()
        
        sequence = self._sequence(subject, action, sub_action, camera)
        BG = self._BG_handle(subject, action, sub_action, camera)
        mask = self._core.getFrame(BG, self._core.double(frame))
        mask = np.reshape(np.asarray(mask._data, dtype=np.float), newshape=(mask._size[1], mask._size[0])).transpose(1, 0)

        return mask
    
    def ToF(self, subject, action, sub_action, frame):
        '''Get a single time-of-flight image.
        
        Params:
            subject: The subject number in 1, 5, 6, 7, 8, 9 and 11.
            action: The action number in the range between 1 and 16.
            sub_action: The sub-action number 1 or 2.
            frame: The frame number.
        
        Return:
            The background mask as numpy array in the height-width shape.
        '''
        if not self.valid_sequence(subject, action, sub_action, camera = 2):
            raise IndexError()
        
        max_frame = self.max_frame(subject, action, sub_action)
        if not (1 <= frame <= max_frame):
            raise IndexError()
        
        ToF = self._ToF_handle(subject, action, sub_action)
        ToF = self._core.getFrame(ToF, self._core.double(frame))
        ToF = np.reshape(np.asarray(ToF._data, dtype=np.float), newshape=(ToF._size[1], ToF._size[0])).transpose(1, 0)

        return ToF
    
    def RGB_video_name(self, subject, action, sub_action, camera):
        '''Get the RGB video file name.
        
        Params:
            subject: The subject number in 1, 5, 6, 7, 8, 9 and 11.
            action: The action number in the range between 1 and 16.
            sub_action: The sub-action number 1 or 2.
            camera: The camera number in the range between 1 and 4.
        
        Return:
            The file name of RGB video.
        '''
        if not self.valid_sequence(subject, action, sub_action, camera):
            raise IndexError()
        
        RGB = self._RGB_handle(subject, action, sub_action, camera)
        var_name = 'RGB_%02d_%02d_%d_%d' % (subject, action, sub_action, camera)
        
        return self._core.eval('%s.Reader.VideoName;' % var_name)
    
    def pose_3D(self, subject, action, sub_action, camera, frame):
        '''Get the keypoint positions.
        
        Params:
            subject: The subject number in 1, 5, 6, 7, 8, 9 and 11.
            action: The action number in the range between 1 and 16.
            sub_action: The sub-action number 1 or 2.
            camera: The camera number in the range between 1 and 4.
            frame: The frame number.
        
        Return:
            32 Keypoint 3D positions.
        '''
        if not self.valid_sequence(subject, action, sub_action, camera):
            raise IndexError()
        
        max_frame = self.max_frame(subject, action, sub_action)
        if not (1 <= frame <= max_frame):
            raise IndexError()
        
        sequence = self._sequence(subject, action, sub_action, camera)
        var_name = 'sequence_%02d_%02d_%d_%d' % (subject, action, sub_action, camera)
        self._core.eval('%s.IdxFrames = %d;' % (var_name, frame), nargout = 0)
    
        pose_3D = self._core.H36MComputeFeatures(sequence, self._core.workspace['features'])[0]
        return np.reshape(np.asarray(pose_3D), newshape=(32, 3))
    
    def valid_sequence(self, subject, action, sub_action, camera):
        '''Check if the sequence is valid.
        
        Params:
            subject: The subject number in 1, 5, 6, 7, 8, 9 and 11.
            action: The action number in the range between 1 and 16.
            sub_action: The sub-action number 1 or 2.
            camera: The camera number in the range between 1 and 4.
            frame: The frame number.
        
        Return:
            True if the valid sequence.
            Otherwise False.
        '''
        return subject in H36M.subjects and\
            1 <= action <= 16 and\
            1 <= sub_action <= 2 and\
            1 <= camera <= 4
    
    def max_frame(self, subject, action, sub_action):
        '''Get the maximum frame.
        
        Params:
            subject: The subject number in 1, 5, 6, 7, 8, 9 and 11.
            action: The action number in the range between 1 and 16.
            sub_action: The sub-action number 1 or 2.
        
        Return:
            The maximum frame.
        '''
        return self._core.getNumFrames(self._core.workspace['DB'], subject, action, sub_action)
    
    def clear_workspace(self):
        '''Clear the MATLAB workspace.
        '''
        self._core.eval('clear; close all;', nargout = 0)
    
    @lru_cache(maxsize = 1024)
    def _sequence(self, subject, action, sub_action, camera):
        '''Covert to sequence_object.
        
        Params:
            subject: The subject number in 1, 5, 6, 7, 8, 9 and 11.
            action: The action number in the range between 1 and 16.
            sub_action: The sub-action number 1 or 2.
            camera: The camera number in the range between 1 and 4.
        
        Return:
            H36MSequence object.
        '''
        var_name = 'sequence_%02d_%02d_%d_%d' % (subject, action, sub_action, camera)
        self._core.workspace[var_name] = self._core.H36MSequence(subject, action, sub_action, camera, -1)
        
        return self._core.workspace[var_name]
    
    @lru_cache(maxsize = 1024)
    def _RGB_handle(self, subject, action, sub_action, camera):
        '''Get RGB video handle object.
        
        Params:
            subject: The subject number in 1, 5, 6, 7, 8, 9 and 11.
            action: The action number in the range between 1 and 16.
            sub_action: The sub-action number 1 or 2.
            camera: The camera number in the range between 1 and 4.
        
        Return:
            The MATLAB VideoPlayer.
        '''
        var_name = 'RGB_%02d_%02d_%d_%d' % (subject, action, sub_action, camera)
        sequence = self._sequence(subject, action, sub_action, camera)
        self._core.workspace[var_name] = self._core.serializer(self._core.workspace['feature_RGB'], sequence)
        
        return self._core.workspace[var_name]
    
    @lru_cache(maxsize = 1024)
    def _BB_handle(self, subject, action, sub_action, camera):
        '''Get the bounding-box handle object.
        
        Params:
            subject: The subject number in 1, 5, 6, 7, 8, 9 and 11.
            action: The action number in the range between 1 and 16.
            sub_action: The sub-action number 1 or 2.
            camera: The camera number in the range between 1 and 4.
        
        Return:
            The MATLAB VideoPlayer.
        '''
        var_name = 'BB_%02d_%02d_%d_%d' % (subject, action, sub_action, camera)
        sequence = self._sequence(subject, action, sub_action, camera)
        self._core.workspace[var_name] = self._core.serializer(self._core.workspace['feature_BB'], sequence)
        
        return self._core.workspace[var_name]
    
    @lru_cache(maxsize = 1024)
    def _BG_handle(self, subject, action, sub_action, camera):
        '''Get the background handle object.
        
        Params:
            subject: The subject number in 1, 5, 6, 7, 8, 9 and 11.
            action: The action number in the range between 1 and 16.
            sub_action: The sub-action number 1 or 2.
            camera: The camera number in the range between 1 and 4.
        
        Return:
            The MATLAB VideoPlayer.
        '''
        var_name = 'BG_%02d_%02d_%d_%d' % (subject, action, sub_action, camera)
        sequence = self._sequence(subject, action, sub_action, camera)
        self._core.workspace[var_name] = self._core.serializer(self._core.workspace['feature_BG'], sequence)
        
        return self._core.workspace[var_name]
    
    @lru_cache(maxsize = 256)
    def _ToF_handle(self, subject, action, sub_action):
        '''Get the time-of-flight handle object.
        
        Params:
            subject: The subject number in 1, 5, 6, 7, 8, 9 and 11.
            action: The action number in the range between 1 and 16.
            sub_action: The sub-action number 1 or 2.
        
        Return:
            The ToF cdf file wrapper.
        '''
        var_name = 'ToF_%02d_%02d_%d' % (subject, action, sub_action)
        self._core.workspace[var_name] = self._core.H36MTOFDataAccess(subject, action, sub_action)
        
        return self._core.workspace[var_name]

In [3]:
query = H36M()

In [4]:
subject = 1
action = 2
sub_action = 1
camera = 4
frame = query.max_frame(subject, action, sub_action)
frame = 751

In [ ]:
cnt = 0
for subject in [1]:
    for action in range(2, 16+1):
        for sub_action in [1, 2]:
            for camera in [1, 2, 3, 4]:
                name = query.RGB_video_name(subject, action, sub_action, camera)
                # name = basename(name)[:-4].replace(' ', '_')
                
                if (int(query.max_frame(subject, action, sub_action)) - 1) % 5 == 0:
                    cnt = cnt + 1
                
                continue
                
                for frame in tqdm(range(1, int(query.max_frame(subject, action, sub_action)) + 1, 5)):
                    '''
                    file = 'S%d_%s_%05d' % (subject, name, frame)
                    print(file)
                    break
                    '''
                    
                    name = 'tmp/RGB_%02d_%02d_%d_%d_%04d.png' % (subject, action, sub_action, camera, frame)
                    if exists(name):
                        continue
                    image = query.RGB_image(subject, action, sub_action, camera, frame)
                    imageio.imwrite(name, np.asarray(255 * image, dtype=np.uint8))
print(cnt)

In [ ]:
name = query.RGB_video_name(subject, action, sub_action, camera)
print('Video at: %s' % name)

In [ ]:
dst = query.undistorted_RGB_image(subject, action, sub_action, camera, frame)
imageio.imwrite('tmp/dst.png', dst)

In [ ]:
image = query.RGB_image(subject, action, sub_action, camera, frame)
imageio.imwrite('tmp/original.png', image)

In [ ]:
pose = query.pose_3D(subject, action, sub_action, camera, frame)
print(pose.shape)
pelvis = [1]
left_leg = [7, 8, 9]
right_leg = [2, 3, 4]
spine = [13, 14, 15, 16]
left_arm = [18, 19, 20]
right_arm = [26, 27, 28]
for idx in pelvis + left_leg + right_leg + spine + left_arm + right_arm:
    print('[%f, %f, %f],' % (pose[idx - 1][0], pose[idx - 1][1], pose[idx - 1][2]))

In [ ]:
f, c, k, p = query.camera_intrinsics(subject, action, sub_action, camera, raw_data=True)
print(f)
print(c)
print(k)
print(p)

In [6]:
for camera in range(1, 4 + 1):
    sequence = query._sequence(subject, action, sub_action, camera)
    query.workspace['cam'] = query._core.getCamera(sequence)
    f, c, k, p = query.camera_intrinsics(subject, action, sub_action, camera, raw_data=True)
    cam = query._core.eval('cam.Name')
    print(cam)
    np.savetxt('calibration/%s_f.txt' % cam, f)
    np.savetxt('calibration/%s_c.txt' % cam, c)
    np.savetxt('calibration/%s_k.txt' % cam, k)
    np.savetxt('calibration/%s_p.txt' % cam, p)

54138969
55011271
58860488
60457274


In [ ]:
def project(keypoints, f, c, k, p):
    X = keypoints.transpose(1, 0) # Already in 3D pose
    XX = np.divide(X[0:2, :], X[2, :])
    r2 = np.power(XX[0, :], 2) + np.power(XX[1, :], 2)
    radial = np.dot(k, np.asarray([r2, np.power(r2, 2), np.power(r2, 3)])) + 1
    tan = p[0] * XX[1, :] + p[1] * XX[0, :]
    temp = radial + tan
    first = XX * np.stack([temp, temp])
    second = np.expand_dims(np.asarray([p[1], p[0]]), axis=1) * np.expand_dims(r2, axis=0)
    XXX = first + second
    XXX = XXX.transpose(1, 0)
    proj = f * XXX + c
    
    return proj

In [ ]:
capture_box_length = 2400
capture_box_half_length = capture_box_length / 2

root = query.pose_3D(subject, action, sub_action, camera, frame)[0, :]

x_axis = np.asarray([1, 0, -root[0]/root[2]])
y_axis = np.cross(root, x_axis)
x_axis = x_axis / np.linalg.norm(x_axis)
y_axis = y_axis / np.linalg.norm(y_axis)
z_axis = root / np.linalg.norm(root)

broadcasting = np.asarray([
    [0, 0, 0],
    [1, 1, 1],
    [1, 1, -1],
    [1, -1, 1],
    [1, -1, -1],
    [-1, 1, 1],
    [-1, 1, -1],
    [-1, -1, 1],
    [-1, -1, -1],
])
capture_box = np.asarray([
    root
    + capture_box_half_length * multiplier[0] * x_axis
    + capture_box_half_length * multiplier[1] * y_axis
    + capture_box_half_length * multiplier[2] * z_axis for multiplier in broadcasting])

In [ ]:
cb = project(np.append(capture_box, pose, axis=0), f, c, k, p)

image = query.RGB_image(subject, action, sub_action, camera, frame)
for point in cb:
    x, y = point.astype(int)
    for tx in range(-5, 5):
        for ty in range(-5, 5):
            xx = x + tx
            yy = y + ty
            if not 0 <= xx < image.shape[1] or not 0 <= yy < image.shape[0]:
                continue
            image[yy, xx, :] = [1, 0, 0]
imageio.imwrite('projected.png', image)

distort, cam_mat = query.camera_intrinsics(subject, action, sub_action, camera)

h,  w = image.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(cam_mat,distort,(w,h),1,(w,h))

# undistort
dst = cv2.undistort(image, cam_mat, distort, None, newcameramtx)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
imageio.imwrite('projected_distorted.png', dst)

In [ ]:
print(cb)

In [ ]:
pose = query.pose_3D(subject, action, sub_action, camera, frame)
for idx, keypoint in enumerate(pose):
    if idx == 0:
        continue
    pose[idx] = keypoint - pose[0]
pose[0, :] = [0, 0, 0]
pose_backup = copy.deepcopy(pose)
for idx, keypoint in enumerate(pose):
    pose[idx] = [x_axis[0] * keypoint[0] + y_axis[0] * keypoint[1] + z_axis[0] * keypoint[2],
                 x_axis[1] * keypoint[0] + y_axis[1] * keypoint[1] + z_axis[1] * keypoint[2],
                 x_axis[2] * keypoint[0] + y_axis[2] * keypoint[1] + z_axis[2] * keypoint[2],]

x_axis = 0
y_axis = 2
z_axis = 1

pose[:, z_axis] = -pose[:, z_axis]
pose_backup[:, z_axis] = -pose_backup[:, z_axis]

In [ ]:
query._core.eval('close all;', nargout=0)
query._core.workspace['fig'] = query('figure(1);')
query._core.eval('view(180, 90);', nargout=0)
# query('axis([-2000, 2000, -2000, 2000, 0, 2000])', exe_only=True)
# query._core.eval('hold on;', nargout=0)

spine = np.array([pose[x] for x in [0, 11, 12, 13, 14, 15]])
query._core.plot3(
    matlab.double(spine[:, x_axis].tolist()),
    matlab.double(spine[:, y_axis].tolist()),
    matlab.double(spine[:, z_axis].tolist()),
    'Color', 'red', nargout=0)
query._core.eval('hold on;', nargout=0)

right_leg = np.array([pose[x] for x in [0, 1, 2, 3, 4]])
query._core.plot3(
    matlab.double(right_leg[:, x_axis].tolist()),
    matlab.double(right_leg[:, y_axis].tolist()),
    matlab.double(right_leg[:, z_axis].tolist()),
    'Color', 'green', nargout=0)
query._core.eval('hold on;', nargout=0)

left_leg = np.array([pose[x] for x in [0, 6, 7, 8, 9]])
query._core.plot3(
    matlab.double(left_leg[:, x_axis].tolist()),
    matlab.double(left_leg[:, y_axis].tolist()),
    matlab.double(left_leg[:, z_axis].tolist()),
    'Color', 'blue', nargout=0)
query._core.eval('hold on;', nargout=0)

left_arm = np.array([pose[x] for x in [12, 16, 17, 18, 19, 21]])
query._core.plot3(
    matlab.double(left_arm[:, x_axis].tolist()),
    matlab.double(left_arm[:, y_axis].tolist()),
    matlab.double(left_arm[:, z_axis].tolist()),
    'Color', 'blue', nargout=0)
query._core.eval('hold on;', nargout=0)

right_arm = np.array([pose[x] for x in [12, 24, 25, 26, 27, 28, 30]])
query._core.plot3(
    matlab.double(right_arm[:, x_axis].tolist()),
    matlab.double(right_arm[:, y_axis].tolist()),
    matlab.double(right_arm[:, z_axis].tolist()),
    'Color', 'green', nargout=0)
query._core.eval('hold on;', nargout=0)

query._core.eval('grid on;', nargout=0)
query._core.eval('axis([-1200, 1200, -1200, 1200, -1200, 1200]);', nargout=0)

In [ ]:
query._core.workspace['fig2'] = query('figure(2);')
query._core.eval('view(180, 90);', nargout=0)
# query('axis([-2000, 2000, -2000, 2000, 0, 2000])', exe_only=True)
# query._core.eval('hold on;', nargout=0)

spine = np.array([pose_backup[x] for x in [0, 11, 12, 13, 14, 15]])
query._core.plot3(
    matlab.double(spine[:, x_axis].tolist()),
    matlab.double(spine[:, y_axis].tolist()),
    matlab.double(spine[:, z_axis].tolist()),
    'Color', 'red', nargout=0)
query._core.eval('hold on;', nargout=0)

right_leg = np.array([pose_backup[x] for x in [0, 1, 2, 3, 4]])
query._core.plot3(
    matlab.double(right_leg[:, x_axis].tolist()),
    matlab.double(right_leg[:, y_axis].tolist()),
    matlab.double(right_leg[:, z_axis].tolist()),
    'Color', 'green', nargout=0)
query._core.eval('hold on;', nargout=0)

left_leg = np.array([pose_backup[x] for x in [0, 6, 7, 8, 9]])
query._core.plot3(
    matlab.double(left_leg[:, x_axis].tolist()),
    matlab.double(left_leg[:, y_axis].tolist()),
    matlab.double(left_leg[:, z_axis].tolist()),
    'Color', 'blue', nargout=0)
query._core.eval('hold on;', nargout=0)

left_arm = np.array([pose_backup[x] for x in [12, 16, 17, 18, 19, 21]])
query._core.plot3(
    matlab.double(left_arm[:, x_axis].tolist()),
    matlab.double(left_arm[:, y_axis].tolist()),
    matlab.double(left_arm[:, z_axis].tolist()),
    'Color', 'blue', nargout=0)
query._core.eval('hold on;', nargout=0)

right_arm = np.array([pose_backup[x] for x in [12, 24, 25, 26, 27, 28, 30]])
query._core.plot3(
    matlab.double(right_arm[:, x_axis].tolist()),
    matlab.double(right_arm[:, y_axis].tolist()),
    matlab.double(right_arm[:, z_axis].tolist()),
    'Color', 'green', nargout=0)
query._core.eval('hold on;', nargout=0)

query._core.eval('grid on;', nargout=0)
query._core.eval('axis([-1200, 1200, -1200, 1200, -1200, 1200]);', nargout=0)

In [ ]:
print('Vector3[] positions = new [] { ')
for part in [right_arm]: #spine, right_arm, left_arm, right_leg, right_leg]:
    for point in part * 0.03:
        print('\tnew Vector3(%.4ff, %.4ff, %.4ff), ' % (point[0], point[1], point[2]))
print('};')

In [ ]:
bb = query.BB_mask(subject, action, sub_action, camera, frame)
imageio.imwrite('tmp/bounding_box.png', bb)

In [ ]:
ul = [-1, -1]
for y in range(0, bb.shape[0]):
    for x in range(0, bb.shape[1]):
        if bb[y, x] != 0:
            ul = [x, y]
            break
    if ul != [-1, -1]:
        break
        
br = [-1, -1]
for y in range(bb.shape[0] - 1, -1, -1):
    for x in range(bb.shape[1] - 1, -1, -1):
        if bb[y, x] != 0:
            br = [x, y]
            break
    if br != [-1, -1]:
        break

In [ ]:
print(ul, br)
print((np.asarray(ul) + np.asarray(br))/2)

In [ ]:
# for part_idx in tqdm(range(len(annotation['part'][image_idx]))):
# x, y = annotation['part'][image_idx][part_idx].astype(int)
image = query.RGB_image(subject, action, sub_action, camera, frame)

for y in range(0, bb.shape[0]):
    for x in range(0, bb.shape[1]):
        if bb[y, x] != 0:
            image[y, x] = image[y, x] * 0.5 + [0, 0.5, 0]

x, y = ul
for tx in range(-5, 5):
    for ty in range(-5, 5):
        xx = x + tx
        yy = y + ty
        if not 0 <= xx < image.shape[1] or not 0 <= yy < image.shape[0]:
            continue
        image[yy, xx, :] = [1, 0, 0]
        
x, y = br
for tx in range(-5, 5):
    for ty in range(-5, 5):
        xx = x + tx
        yy = y + ty
        if not 0 <= xx < image.shape[1] or not 0 <= yy < image.shape[0]:
            continue
        image[yy, xx, :] = [1, 0, 0]

x, y = [ 387, 367]# [ 499, 462]# [ 466, 465]
for tx in range(-5, 5):
    for ty in range(-5, 5):
        xx = x + tx
        yy = y + ty
        if not 0 <= xx < image.shape[1] or not 0 <= yy < image.shape[0]:
            continue
        image[yy, xx, :] = [0, 0, 1]
        
imageio.imwrite('test.png', image)

In [ ]:
bg = query.BG_mask(subject, action, sub_action, camera, frame)
imageio.imwrite('bg.png', bg)

In [ ]:
tof = query.ToF(subject, action, sub_action, frame)
tof /= np.max(tof)
imageio.imwrite('tof.png', tof)

engine.workspace['Features'] = [
    engine.H36MPose3DPositionsFeature(),
    engine.H36MPose2DPositionsFeature(),
    engine.H36MPose3DPositionsFeature('Monocular', True),
]

engine.workspace['Sequence'] = engine.H36MSequence(query.subject, query.action, query.sub_action, query.camera, query.frame)
engine.workspace['F'] = engine.H36MComputeFeatures(engine.workspace['Sequence'], engine.workspace['Features'])

engine.workspace['c'] = engine.getCamera(engine.workspace['Sequence'])

engine.workspace['projected'] = engine.project(engine.workspace['c'], engine.workspace['F'][2])

engine.workspace['camera0'] = engine.H36MCamera(engine.workspace['DB'], 0, 1)
engine.workspace['projected'] = engine.project(engine.workspace['camera0'], engine.workspace['F'][2])